In [15]:
import numpy as np
import os
import miga_emblaze.emblaze as m_emb
from miga_emblaze.emblaze.utils import Field, ProjectionTechnique
import cv2

embedding_dir =  '/rhome/lianyu.zhou/cache/cluster_test_set_gformer_l3_b256_atom60'
need = ['img', 'graph']
modal_num = len(need)
samples_num = 3000
embeddings_list = [np.load(os.path.join(embedding_dir, f"{name}_embedding.npy"))[:samples_num]
                                  for name in need]

In [9]:
show_img = True

In [16]:
modal_imgs_list = [[] for _ in range(modal_num)]
for m_i, modal_name in enumerate(need):
    modal_imgs_dir = f'/rhome/lianyu.zhou/dataset/omics_{modal_name}_vis'
    for i in range(samples_num):
        read_path = os.path.join(modal_imgs_dir, f"{i:05d}.png")
        img = cv2.imread(read_path)[..., ::-1]
        modal_imgs_list[m_i].append(img)
merged_modal_imgs_list = modal_imgs_list[0] + modal_imgs_list[1]

In [17]:
if show_img:
    thumbnails = m_emb.ImageThumbnails(merged_modal_imgs_list)
else:
    thumbnails = None

In [18]:
import umap
color_list = [i * np.ones(e.shape[0]) for i, e in enumerate(embeddings_list)]
color_array, merge_embedding = list(map(lambda x: np.concatenate(x, axis=0), [color_list, embeddings_list]))
trans = umap.UMAP(metric='cosine',n_neighbors=100).fit(merge_embedding)
reduced_embedding = trans.embedding_
view_embedding = m_emb.EmbeddingSet([m_emb.Embedding({Field.POSITION: reduced_embedding, Field.COLOR: color_array})])
view_embedding.compute_neighbors(metric='euclidean', n_neighbors=100)
w = m_emb.Viewer(embeddings=view_embedding, thumbnails=thumbnails)
w

Viewer(colorScheme='tableau', data={'data': [{'_format': 'compressed', '_idtype': 'u2', '_length': 6000, 'ids'…

In [19]:
from utils.utils import add_new_embedding
new_reduced, new_thum = add_new_embedding([i for i in range(samples_num, samples_num + 20)], 
                                          0, 
                                          trans, 
                                          reduced_embedding, 
                                          color_array, 
                                          merged_modal_imgs_list)


In [21]:
new_w = m_emb.Viewer(embeddings=new_reduced, thumbnails=new_thum)
new_w
